In [183]:
import os, sys
import re
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np
from collections import defaultdict
import nltk
import math
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.datasets import load_iris
from sklearn import tree
import datetime
import time;
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import pandas
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import io

# iccid,imei,imsi, mac address, android id, android advertiser id, device_id
# name , gender, dob, relation ship status, email address, phone number, address book infromation
# GPS lattitude longitude, zip code
# username, password

In [184]:
import randomForest as rf

SyntaxError: unexpected EOF while parsing (randomForest.py, line 158)

In [ ]:
def getEmbeddingMatrix(wordIndex):
    """Populate an embedding matrix using a word-index. If the word "happy" has an index 19,
       the 19th row in the embedding matrix should contain the embedding vector for the word "happy".
    Input:
        wordIndex : A dictionary of (word : index) pairs, extracted using a tokeniser
    Output:
        embeddingMatrix : A matrix where every row has 100 dimensional GloVe embedding
    """
    embeddingsIndex = {}
    # Load the embedding vectors from ther GloVe file
    with io.open(os.path.join("./", 'glove.6B.100d.txt'), encoding="utf8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            embeddingVector = np.asarray(values[1:], dtype='float32')
            embeddingsIndex[word] = embeddingVector
    
    print('Found %s word vectors.' % len(embeddingsIndex))
    
    # Minimum word index of any word is 1. 
    embeddingMatrix = np.zeros((len(wordIndex) + 1, EMBEDDING_DIM))
    for word, i in wordIndex.items():
        embeddingVector = embeddingsIndex.get(word)
        if embeddingVector is not None:
            # words not found in embedding index will be all-zeros.
            embeddingMatrix[i] = embeddingVector
    
    return embeddingMatrix

In [ ]:
class NaiveBayes:
    def __init__(self):
        self.d0 = defaultdict(lambda:0)
        self.d1 = defaultdict(lambda:0)
        self.d = defaultdict(lambda:0)
        self.prior0 = 0.0
        self.prior1 = 0.0
        self.c0 = 0
        self.c1 = 0
    def train(self,text,label):
        
        self.prior0 = len(label[label==0]) / len(label)
        self.prior1 = len(label[label==1]) / len(label)  
        #print(self.prior0,'\n',self.prior1)
        
        i=0
        for t in text:
            if label[i]==0:
                for w in t:
                    self.d0[w]+=1
                    self.d[w]+=1
                    self.c0+=1
            else:
                for w in t:
                    self.d1[w]+=1
                    self.d[w]+=1
                    self.c1+=1
            i+=1
                    
    def test(self,texts):
        
        l = []
        vocab = len(self.d)
        for words in texts:
            #print("\n\npacket:\n",words)
            L0 = 0.0
            L1 = 0.0
            for w in words:
                #print ("WORD: ",w)
                L0 += math.log( (self.d0[w]+1) / ( self.c0 + vocab ) )
                L1 += math.log( (self.d1[w]+1) / ( self.c1 + vocab ) )
                #print("0: ",self.d0[w],L0)
                #print('1: ',self.d1[w],L1)
            #print('likeli: ',L0,'  ',L1)
            p0 = L0#self.prior0 * L0
            p1 = L1#self.prior1 * L1
            #print("Final prob: ",p0,'  ',p1)
            if p0>=p1:
                l.append(0)
            else:
                l.append(1)
            #a=input("waiting...")
        return np.array(l)

# Reading data and storing inside pandas dataframe

In [3]:
packets = None
i=0
total=0
for filename in os.listdir('android/'):
    
    #print('============================'+filename+'============================')
    d = pd.read_json('android/'+filename)
    df = pd.DataFrame.from_dict(d)
    df=df.T
    total+=len(df)
    if i==0:
        packets=df
    else:
        packets=packets.append(df)
    i+=1

# Data Visualizations

In [ ]:
unique_domains=(np.unique((packets.domain)))

In [ ]:
c=0
common_domains = []
num_packets = []
for u in unique_domains:
    num_of_packets = len(packets.domain[packets.domain==u])
    if num_of_packets>150:
        common_domains.append(u)
        num_packets.append(num_of_packets)
common_domains = np.array(common_domains)
num_packets = np.array(num_packets)

plt.rcdefaults()
fig, ax = plt.subplots()

ax.barh(common_domains, num_packets, align='center',
        color='green', ecolor='black')
ax.set_xlabel('Number of Packets')
ax.set_title('')

plt.show()

In [ ]:
udomains_leaking_pii = (np.unique(packets.domain[packets.o_pii == 1]))
domains_leaking_pii = ((packets.domain[packets.o_pii == 1]))
common_domains_leaking_pii = []
count_of_packets = []
for u in udomains_leaking_pii:
    num_of_packets = len(domains_leaking_pii[domains_leaking_pii==u])
    if num_of_packets>20:
        common_domains_leaking_pii.append(u)
        count_of_packets.append(num_of_packets)
        
plt.rcdefaults()
fig, ax = plt.subplots()

ax.barh(common_domains_leaking_pii, count_of_packets, align='center',color='green', ecolor='black')
ax.set_xlabel('Number of Personal Infromation Leaks')
ax.set_title('')

plt.show()

In [ ]:
piid = defaultdict(lambda:0)
piis = packets.pii_types[packets.o_pii==1]
piis[0][0]
for i in piis:
    for j in i:
        piid[j.lower()]+=1
        
plt.rcdefaults()
fig, ax = plt.subplots()

ax.barh((list(piid.keys())), (list(piid.values())), align='center',color='#539caf', ecolor='black')
ax.set_xlabel('Number of Personal Infromation Leaks')
ax.set_title('')

plt.show()

# Assigning labels to data packets
# Using String Matching

In [4]:
sm_label=[]

In [5]:
for uri in packets.uri:
    uri=uri.lower()
    token = re.split('&',uri)
    chk=re.findall('mac.?address|imei|iccid|imsi|android.?id|advertiser.?id|gender|latitude|longitude|zip.?code|user.?name|password|email|firstname|macaddr|serialnumber',uri)
    chk=np.array(chk)
    chk2=0
    if (len(chk)>0):
        chk2=0
        for pi in np.unique(chk):
            for t in token:
                ind=t.find(pi)+len(pi)
                if ind+8 < len(t):
                    chk2=1
                    break
            if chk2==1:
                break
    if (chk2==1):
        sm_label.append(1)
    else:
        sm_label.append(0)

In [6]:
sm_label=np.array(sm_label)

In [7]:
o_label=[]

In [8]:
for i in range(0,len(packets)):
    row=packets.iloc[i]
    if (row.pii_types==None):
        o_label.append(0)
    else:
        o_label.append(1)

In [9]:
o_label=np.array(o_label)
packets['o_pii']=o_label

In [10]:
stringMatching_cm = confusion_matrix(o_label,sm_label)
stringMatching_cm = pandas.DataFrame(stringMatching_cm, ['Non-PII','PII'], ['Non-PII','PII'])

In [11]:
stringMatching_acc = (len(o_label[o_label==sm_label])/len(o_label))

In [12]:
stringMatching_f1 = f1_score(o_label,sm_label)

# Obtaining Words From Data Packets

In [143]:
def text_to_words(raw_data):
    wrds = []
    tokens = re.split(",|\t|/|\\||\\*|!|#|&|\\?|\n|;|\\{|\\}|\\(|\\)| ",raw_data)
    #print("BEFORE ::::::::: ",tokens)
    for i in range(0,len(tokens)):
        t = tokens[i]
        t=t.strip()
        #print ("TOKEN UTHAYA HA: ",t)
        if (len(t) < 1):
            continue
        t=re.sub("\"|\'|\\[|\\]|http://|https://|ftp://","",t)
        isval = True
        if (':' in t and '=' not in t):
            t.replace(':', '=', 1)
        if ('=' in t):
            tsplit = re.split('=',t)
            #print ("SPLITS HAIN: ", tsplit)
            if len(tsplit)>1 and '' not in tsplit:
                isval = False
                reconk = tsplit[0]
                reconv = tsplit[1].strip()
                #print("P YA HA:",reconv)
                if (reconv[-1]==':' or reconv[-1]=='-' or reconv[-1]=='='):
                    reconv = reconv[0:len(reconv)]
                #print("YA HA:",reconv)
                if (reconv[0]=='>'):
                    reconv = reconv[1:]
                if len(reconk)>=1:
                    wrds.append(reconk)
                if len(reconv)>=1:
                    wrds.append(reconv)
            elif len(tsplit) == 1:
                t = tsplit[0]
        if (isval):
            t=t.strip()
            if (len(t)==0):
                continue
            if len(t) == 1:
                tmpt = t[0]
                if (tmpt.isdigit==False and tmpt.isalpha==False):
                    continue
            wrds.append(t)
        tokens[i]=t
    return wrds

In [144]:
text = []
tokenize_packets = []
label = []
for i in range(0,len(packets)):
    row=packets.iloc[i]
    words = []
    words = text_to_words(row.uri.lower())
    text.extend(words)
    tokenize_packets.append(words)
    label.append(row.o_pii)

In [145]:
tokenize_packets = np.array(tokenize_packets)
label = np.array(label)

In [146]:
dic_pii = defaultdict(lambda:0)
dic_nonpii = defaultdict(lambda:0)

In [147]:
i=0
for p in tokenize_packets:
    if (label[i]==0):
        for w in p:
            dic_nonpii[w]+=1
    i+=1

# Using NaiveBayes

In [ ]:
nb = NaiveBayes()
nb.train(tokenize_packets[0:9800],label[0:9800])

In [ ]:
pl = nb.test(tokenize_packets[9800:])

In [ ]:
naiveBayes_cm = confusion_matrix(label[9800:],pl)
naiveBayes_cm = pandas.DataFrame(naiveBayes_cm, ['Non-PII','PII'], ['Non-PII','PII'])

In [ ]:
otlabel = label[9800:]
naiveBayes_acc = len(otlabel[pl == otlabel])/len(otlabel)

In [ ]:
naiveBayes_f1 = f1_score(otlabel, pl)

# Using Extra Trees

In [148]:
d = defaultdict(lambda:0)
for w in text:
    d[w]+=1

In [149]:
good_words = []
for k in d.keys():
    if d[k]>1:
        good_words.append(k)

In [150]:
len(good_words)

6853

In [151]:
features = []
c=0
for w in good_words:
    f = []
    for p in tokenize_packets:
        if w in p:
            f.append(1)
        else:
            f.append(0)
    features.append(f)
    c+=1

In [152]:
nf = np.array(features)
nf = nf.T

In [153]:
X_train, X_test, y_train, y_test = train_test_split(tokenize_packets, label, test_size=0.25)

In [154]:
X_train.shape

(9809,)

In [155]:
train_features = []
c=0
for w in good_words:
    f = []
    for p in X_train:
        if w in p:
            f.append(1)
        else:
            f.append(0)
    train_features.append(f)
    c+=1
train_features = np.array(train_features)
train_features = train_features.T

In [156]:
test_features = []
c=0
for w in good_words:
    f = []
    for p in X_test:
        if w in p:
            f.append(1)
        else:
            f.append(0)
    test_features.append(f)
    c+=1
test_features = np.array(test_features)
test_features = test_features.T

In [157]:
clf = ExtraTreesClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)

In [158]:
clf.fit(train_features, y_train)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [159]:
predictions = clf.predict(test_features)

In [160]:
extraTrees_acc = len(y_test[y_test==predictions]) / len(y_test)

In [161]:
extraTrees_cm = confusion_matrix(y_test, predictions)
extraTrees_cm = pandas.DataFrame(extraTrees_cm, ['Non-PII','PII'], ['Non-PII','PII'])

In [162]:
extraTrees_f1 = f1_score(y_test, predictions)

In [163]:
extraTrees_f1

0.9568345323741008

In [164]:
extraTrees_cm

,Non-PII,PII
Non-PII,2835,9
PII,27,399


In [165]:
pii_packets = X_test[predictions==1]

# Extracting PIIs

In [176]:
piis_in_packets = []
for i in range(len(pii_packets)):
    piis = []
    for w in pii_packets[i]:
        prob = 0.0
        #print(w)
        prob = dic_nonpii[w] / d[w]
        if (prob<=0.0):
            piis.append(w)
    piis_in_packets.append(piis)
        #print(prob)
        #a=input("RUK JA O DIL DIWANAY\n")

In [181]:
piis_in_packets[4]

['5a666ad479ce6ffa5636b67dae7a2e55', 'design']

# AdaBoost Classifier

In [ ]:
clfada = AdaBoostClassifier(n_estimators=100)

In [ ]:
clfada.fit(X_train, y_train)

In [ ]:
preada=clfada.predict(X_test)

In [ ]:
ada_cm = confusion_matrix(y_test, preada)
ada_cm = pandas.DataFrame(ada_cm, ['Non-PII','PII'], ['Non-PII','PII'])

In [ ]:
ada_f1 = f1_score(y_test, preada)

In [ ]:
ada_acc = len(y_test[y_test==preada]) / len(y_test)

# Gradient Boosted DTs

In [ ]:
clfgtb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=10, random_state=0)

In [ ]:
clfgtb.fit(X_train, y_train)

In [ ]:
pregtb = clfgtb.predict(X_test)

In [ ]:
gbdt_cm = confusion_matrix(y_test, pregtb)
gbdt_cm = pandas.DataFrame(gbdt_cm, ['Non-PII','PII'], ['Non-PII','PII'])

In [ ]:
gbdt_f1 = f1_score(y_test, pregtb)

In [ ]:
gbdt_acc = len(y_test[y_test==pregtb]) / len(y_test)

# Results

In [ ]:
l1 = [naiveBayes_acc,stringMatching_acc,extraTrees_acc,ada_acc,gbdt_acc]
l2 = [naiveBayes_f1,stringMatching_f1,extraTrees_f1,ada_f1,gbdt_f1]
l3=[]
l3.append(l1)
l3.append(l2)
l3=np.array(l3)

In [ ]:
results  = pandas.DataFrame(l3,['Accuracy','F1-measure'],['Naive Bayes','String Matching','Extra Trees',"ADA Boosted Classifier","Gradient Boosted Decsion Tress"])

In [ ]:
results

In [ ]:
print ('\033[1m' + 'Confusion Matrix Naive Bayes')
naiveBayes_cm

In [ ]:
print ('\033[1m' + 'Confusion Matrix String Matching')
stringMatching_cm

In [ ]:
print ('\033[1m' + 'Confusion Matrix Extra Trees')
extraTrees_cm

In [ ]:
print ('\033[1m' + 'Confusion Matrix GBDT')
gbdt_cm

In [ ]:
print ('\033[1m' + 'Confusion Matrix Ada Boosted Classifier')
ada_cm

# Playing with monkey Data Set

In [ ]:
mpackets = None
i=0
total=0
for filename in os.listdir('monkey/'):
    
    #print filename+'============================')
    d = pd.read_json('monkey/'+filename)
    df = pd.DataFrame.from_dict(d)
    df=df.T
    total+=len(df)
    if i==0:
        mpackets=df
    else:
        mpackets=mpackets.append(df,sort=False)
    i+=1

In [ ]:
mo_label=[]
for i in range(0,len(mpackets)):
    row=mpackets.iloc[i]
    if (row.pii_types==None):
        mo_label.append(0)
    else:
        mo_label.append(1)
mo_label=np.array(mo_label)
mpackets['o_pii']=mo_label

In [ ]:
mtokenize_packets = []
mlabel = []
for i in range(0,len(mpackets)):
    row=mpackets.iloc[i]
    a=row.uri.lower()
    a=re.sub('meid','imei',a)
    words = text_to_words(a)
    #print(words)
    mtokenize_packets.append(words)
    mlabel.append(row.o_pii)
mlabel = np.array(mlabel)
mtokenize_packets = np.array(mtokenize_packets)

In [ ]:
mfeatures = []
for w in good_words:
    f = []
    for p in mtokenize_packets:
        if w in p:
            f.append(1)
        else:
            f.append(0)
    mfeatures.append(f)

In [ ]:
mfeatures = np.array(mfeatures)

In [ ]:
mfeatures = mfeatures.T

In [ ]:
mfeatures.shape

In [ ]:
mclf = ExtraTreesClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
mclf.fit(nf,label)

In [ ]:
mpre = mclf.predict(mfeatures)

In [ ]:
mpre = np.array(mpre)

In [ ]:
confusion_matrix(mlabel, mpre)

In [ ]:
f1_score(mlabel, mpre)

In [ ]:
piid = defaultdict(lambda:0)
piis = mpackets.pii_types[mpackets.o_pii==1]
for i in piis:
    for j in i:
        piid[j.lower()]+=1

In [ ]:
a="Asd/''meidasldk"

In [ ]:
re.sub('meid','imei',a)

In [ ]:
a = np.array([1,2,3,4])

In [ ]:
np.append(a,a)